# Proyecto Modelos y Simulación de Sistemas I

In [3]:
import polars as pl


In [5]:
csv_file = "udea-ai-4-eng-20251-pruebas-saber-pro-colombia/train.csv"

### Cargar el csv

In [9]:
train_df = pl.scan_csv(csv_file)

In [12]:
first_two_rows = train_df.head(2).collect()

display(first_two_rows)

ID,PERIODO,ESTU_PRGM_ACADEMICO,ESTU_PRGM_DEPARTAMENTO,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,FAMI_EDUCACIONPADRE,FAMI_TIENELAVADORA,FAMI_TIENEAUTOMOVIL,ESTU_PRIVADO_LIBERTAD,ESTU_PAGOMATRICULAPROPIO,FAMI_TIENECOMPUTADOR,FAMI_TIENEINTERNET.1,FAMI_EDUCACIONMADRE,RENDIMIENTO_GLOBAL,coef_1,coef_2,coef_3,coef_4
i64,i64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,f64,f64,f64,f64
904256,20212,"""ENFERMERIA""","""BOGOTÁ""","""Entre 5.5 millones y menos de …","""Menos de 10 horas""","""Estrato 3""","""Si""","""Técnica o tecnológica incomple…","""Si""","""Si""","""N""","""No""","""Si""","""Si""","""Postgrado""","""medio-alto""",0.322,0.208,0.31,0.267
645256,20212,"""DERECHO""","""ATLANTICO""","""Entre 2.5 millones y menos de …","""0""","""Estrato 3""","""No""","""Técnica o tecnológica completa""","""Si""","""No""","""N""","""No""","""Si""","""No""","""Técnica o tecnológica incomple…","""bajo""",0.311,0.215,0.292,0.264


| Columna                      | Tipo de Variable      | Acción de Codificación                                                                                                                                   |
|------------------------------|-----------------------|----------------------------------------------------------------------------------------------------------------------------------------------------------|
| `RENDIMIENTO_GLOBAL`         | Ordinal (Target)      | Mapear a valores numéricos `0`–`3` según el orden (e.g., `"bajo": 0`, `"medio-bajo": 1`, `"medio-alto": 2`, `"alto": 3`)                                  |
| `FAMI_ESTRATOVIVIENDA`       | Ordinal               | Mapear `"Estrato 1"`…`"Estrato 6"` a valores numéricos `1`…`6`                                                                                           |
| `ESTU_HORASSEMANATRABAJA`    | Ordinal               | Mapear rangos de texto (e.g., `"0"`, `"Menos de 10 horas"`, etc.) a valores numéricos ordinales                                                          |
| `FAMI_TIENEINTERNET`         | Categórica Binaria    | Convertir a `0` (No) / `1` (Si) y castear a `UInt8`                                                                                                      |
| `FAMI_TIENELAVADORA`         | Categórica Binaria    | Convertir a `0` (No) / `1` (Si) y castear a `UInt8`                                                                                                      |
| `FAMI_TIENEAUTOMOVIL`        | Categórica Binaria    | Convertir a `0` (No) / `1` (Si) y castear a `UInt8`                                                                                                      |
| `ESTU_PRIVADO_LIBERTAD`      | Categórica Binaria    | Convertir a `0` (N) / `1` (S) y castear a `UInt8`                                                                                                        |
| `ESTU_PAGOMATRICULAPROPIO`   | Categórica Binaria    | Convertir a `0` (No) / `1` (Si) y castear a `UInt8`                                                                                                      |
| `FAMI_TIENECOMPUTADOR`       | Categórica Binaria    | Convertir a `0` (No) / `1` (Si) y castear a `UInt8`                                                                                                      |
| `FAMI_TIENEINTERNET.1`       | Categórica Binaria    | Convertir a `0` (No) / `1` (Si) y castear a `UInt8` —  **Revisar si es columna duplicada**                                                                  |
| `FAMI_EDUCACIONPADRE`        | Categórica Nominal    | Aplicar One-hot Encoding o Target Encoding                                                                                                               |
| `FAMI_EDUCACIONMADRE`        | Categórica Nominal    | Aplicar One-hot Encoding o Target Encoding                                                                                                               |
| `ESTU_PRGM_ACADEMICO`        | Categórica Nominal    | Aplicar One-hot Encoding o Target Encoding                                                                                                               |
| `ESTU_PRGM_DEPARTAMENTO`     | Categórica Nominal    | Aplicar One-hot Encoding o Target Encoding                                                                                                               |
| `coef_1` … `coef_4`          | Continua              | Mantener como `Float64`                                                                                                                                 |
| Otras columnas (`ID`, `PERIODO`, etc.) | Variables de Identificación | No hacer nada por ahora                                                                                         |


## Columnas Repetidas

Parece que la columna **FAMI_TIENEINTERNET** está repetida

In [15]:
equal_query = (
    train_df
    .select(
        #Seleccionar las columnas que se van a comparar
        (pl.col("FAMI_TIENEINTERNET") == pl.col("FAMI_TIENEINTERNET.1"))
        .all()
        .alias("equal_query")
    )
    .collect()
)

equal_query_result = equal_query["equal_query"][0]

print(f"Son todos los varlores 'FAMI_TIENEINTERNET' iguales a 'FAMI_TIENEINTERNET.1'? {equal_query_result}")

Son todos los varlores 'FAMI_TIENEINTERNET' iguales a 'FAMI_TIENEINTERNET.1'? True


Vemos que, como efectivamente **está repetida**, la podemos borrar.

In [ ]:
train_df = train_df.drop("FAMI_TIENEINTERNET.1")

## Estandarización

### Ciudades

In [16]:
departments_count = (
    train_df
    .select(
        pl.col("ESTU_PRGM_DEPARTAMENTO")
          .n_unique()
          .alias("n_departamentos")
    )
    .collect()
)["n_departamentos"][0]
print(f"Hay {departments_count} departamentos distintos.")

Hay 31 departamentos distintos.


In [20]:
departaments = (
    train_df
    .select(
        pl.col("ESTU_PRGM_DEPARTAMENTO")
          .unique()
          .sort()
          .alias("departamentos")
    )
    .collect()
)["departamentos"]

# Convertimos la serie de departamentos a una lista
lista_departamentos = departaments.to_list()
print(f"{len(lista_departamentos)} departamentos encontrados:")
# Un salto de línea para mejor legibilidad
print("\n".join(lista_departamentos))

31 departamentos encontrados:
AMAZONAS
ANTIOQUIA
ARAUCA
ATLANTICO
BOGOTÁ
BOLIVAR
BOYACA
CALDAS
CAQUETA
CASANARE
CAUCA
CESAR
CHOCO
CORDOBA
CUNDINAMARCA
GUAVIARE
HUILA
LA GUAJIRA
MAGDALENA
META
NARIÑO
NORTE SANTANDER
PUTUMAYO
QUINDIO
RISARALDA
SAN ANDRES
SANTANDER
SUCRE
TOLIMA
VALLE
VAUPES


### Programas

In [ ]:
unique_programs = (
    train_df
    .select(
        pl.col("ESTU_PRGM_ACADEMICO")
          .str.replace(r'^"|"$', '', literal=False)           # quita comillas al inicio/final
          .str.replace(r'^\s+|\s+$', '', literal=False)  # quita espacios al inicio/final
          .str.to_uppercase()                                 
    )
    .unique()
    .sort("ESTU_PRGM_ACADEMICO")
    .collect()
    .get_column("ESTU_PRGM_ACADEMICO")
    .to_list()
)


#### Distancia de Levenshtein para Programas académicos

In [31]:
def levenshtein(a: str, b: str) -> int:
    if len(a) < len(b):
        a, b = b, a
    if not b:
        return len(a)
    prev = list(range(len(b) + 1))
    for i, ca in enumerate(a, start=1):
        curr = [i]
        for j, cb in enumerate(b, start=1):
            ins = prev[j] + 1
            rem = curr[j-1] + 1
            rep = prev[j-1] + (ca != cb)
            curr.append(min(ins, rem, rep))
        prev = curr
    return prev[-1]

In [32]:
threshold = 2
suggestions = []
n = len(unique_programs)
for i in range(n):
    for j in range(i+1, n):
        p1, p2 = unique_programs[i], unique_programs[j]
        d = levenshtein(p1.lower(), p2.lower())
        if d <= threshold:
            suggestions.append((p1, p2, d))


In [33]:
# Imprimimos las sugerencias de corrección
print(f"\nSugerencias de corrección (distancia <= {threshold}):")
for p1, p2, d in suggestions:
    print(f"{p1} <-> {p2} (distancia: {d})")


Sugerencias de corrección (distancia <= 2):
ADMINISTRACION  FINANCIERA <-> ADMINISTRACION FINANCIERA (distancia: 1)
ADMINISTRACION  FINANCIERA <-> ADMINISTRACIÓN FINANCIERA (distancia: 2)
ADMINISTRACION COMERCIAL <-> ADMINISTRACIÓN COMERCIAL (distancia: 1)
ADMINISTRACION DE COMERCIO EXTERIOR <-> ADMINISTRACIÓN DE COMERCIO EXTERIOR (distancia: 1)
ADMINISTRACION DE EMPRESAS <-> ADMINISTRACI¿N DE EMPRESAS (distancia: 1)
ADMINISTRACION DE EMPRESAS <-> ADMINISTRACIÓN DE EMPRESAS (distancia: 1)
ADMINISTRACION DE EMPRESAS AGROINDUSTRIALES <-> ADMINISTRACIÓN DE EMPRESAS AGROINDUSTRIALES (distancia: 1)
ADMINISTRACION DE EMPRESAS AGROPECUARIAS <-> ADMINISTRACIÓN DE EMPRESAS AGROPECUARIAS (distancia: 1)
ADMINISTRACION DE EMPRESAS TURISTICA <-> ADMINISTRACION DE EMPRESAS TURISTICAS (distancia: 1)
ADMINISTRACION DE EMPRESAS TURISTICA <-> ADMINISTRACIÓN DE EMPRESAS TURISTICAS (distancia: 2)
ADMINISTRACION DE EMPRESAS TURISTICAS <-> ADMINISTRACIÓN DE EMPRESAS TURISTICAS (distancia: 1)
ADMINISTRACION

Evidenciamos que los principales problemas son con tildes y espacios intermedios. Vamos a eliminarlos

In [ ]:
accent_map = {
    "á": "a", "é": "e", "í": "i", "ó": "o", "ú": "u",
    "Á": "A", "É": "E", "Í": "I", "Ó": "O", "Ú": "U",
    "ñ": "n", "Ñ": "N"
}

# Construimos la expresión de normalización
expr = (
    pl.col("ESTU_PRGM_ACADEMICO")
      # 1) quitar comillas
      .str.replace_all('"', '', literal=True)
      # 2) pasar a mayúsculas
      .str.to_uppercase()
)

# 3) encadenar remap de cada tilde
for orig, repl in accent_map.items():
    expr = expr.str.replace_all(orig, repl, literal=True)
"
# 4) eliminar cualquier carácter que no sea A–Z, dígitos o espacio
expr = expr.str.replace_all(r'[^A-Z0-9 ]', '', literal=False)

# 5) colapsar múltiples espacios
expr = expr.str.replace_all(r'\s{2,}', ' ', literal=False)

# 6) quitar espacios al inicio/fin
expr = expr.str.replace_all(r'^\s+|\s+$', '', literal=False)

# Renombramos
expr = expr.alias("prog_norm")

# Ejecutamos lazy y obtenemos la lista limpia:
normalized_programs = (
    train_df
    .select(expr)
    .unique()
    .sort("prog_norm")
    .collect()
    .get_column("prog_norm")
    .to_list()
)

print(f"{len(normalized_programs)} programas normalizados:")
print(normalized_programs[:20], "…")

752 programas normalizados:
['3 CICLO PROFESIONAL NEGOCIOS INTERNACIONALES', 'ACTIVIDAD FISICA Y DEPORTE', 'ACUICULTURA', 'ADMINISTRACIN DE EMPRESAS', 'ADMINISTRACIN DE NEGOCIOS INTERNACIONALES', 'ADMINISTRACIN LOGSTICA', 'ADMINISTRACIN PBLICA', 'ADMINISTRACION', 'ADMINISTRACION AERONAUTICA', 'ADMINISTRACION AGROINDUSTRIAL', 'ADMINISTRACION AGROPECUARIA', 'ADMINISTRACION AMBIENTAL', 'ADMINISTRACION AMBIENTAL Y DE LOS RECURSOS NATURALES', 'ADMINISTRACION BANCARIA Y FINANCIERA', 'ADMINISTRACION COMERCIAL', 'ADMINISTRACION COMERCIAL Y DE MERCADEO', 'ADMINISTRACION COMERCIAL Y FINANCIERA', 'ADMINISTRACION DE AGRONEGOCIOS', 'ADMINISTRACION DE COMERCIO EXTERIOR', 'ADMINISTRACION DE EMPRESAS'] …


## Normalización de columnas Tipo Texto